In [1]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis")
classifier(
    [
        "I've been waiting for a HuggingFace course my whole life.",
        "I hate this so much!",
    ]
)

/mnt/c4a1c56d-0589-48cd-9717-f6c33a86fa3b/dataScience/learn_LLM/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


[{'label': 'POSITIVE', 'score': 0.9598048329353333},
 {'label': 'NEGATIVE', 'score': 0.9994558691978455}]

## 토크나이저를 이용한 전처리

In [2]:
from transformers import AutoTokenizer

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

raw_inputs = ["I've been waiting for long time", "I hate this so much!"]

inputs = tokenizer(raw_inputs, padding=True, truncation=True, return_tensors="pt")
print(input)

<bound method Kernel.raw_input of <ipykernel.ipkernel.IPythonKernel object at 0x7023224c42b0>>


## 모델 살펴보기


In [3]:
from transformers import AutoModel

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModel.from_pretrained(checkpoint)

outputs = model(**inputs)
print(outputs.last_hidden_state.shape)

torch.Size([2, 10, 768])


모델 헤드는 hidden states의 고차원 벡터를 입력으로 받아 다른 차원에 투영합니다. 일반적으로 헤드는 하나 또는 몇 개의 선형 레이어로 구성됩니다. 

## 2장 요약

토크나이저의 작동 방식과 토큰화, 입력 식별자로의 변환, 패딩, 절단 및 어텐션 마스크등을 살펴보았습니다.

In [4]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

sequences = ["I've been waiting for a HuggingFace course my whole life.", "So have I!"]


# 해당 시퀀스를 리스트 내의 최대 시퀀스 길이까지 패딩(padding) 합니다.
# model_inputs = tokenizer(sequences, padding="longest")
# 시퀀스를 모델 최대 길이(model max length)까지 패딩(padding) 합니다.
# (512 for BERT or DistilBERT)
# model_inputs = tokenizer(sequences, padding="max_length")
tokens = tokenizer(sequences, padding=True, truncation=True, return_tensors="pt")
output = model(**tokens)
print(output)

SequenceClassifierOutput(loss=None, logits=tensor([[-1.5607,  1.6123],
        [-3.6183,  3.9137]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)


# 3장 파인 튜닝

기존에 사전 학습된 모델을 사용해 예측을 수행하는 방법을 살펴보았고, 여러분들이 가지고 있는 데이터셋을 가지고 파인튜닝을 하려면 어떻게 해야 할까요?

이 섹션에서는 William B. Dolan과 Chris Brockett의 논문에서 소개된 MRPC(Microsoft Research Paraphrase Corpus) 데이터셋을 예제로 사용할 것입니다. 이 데이터셋은 5,801건의 문장 쌍으로 구성되어 있으며 각 문장 쌍의 관계가 의역(paraphrasing) 관계인지 여부를 나타내는 레이블이 존재합니다(즉, 두 문장이 동일한 의미인지 여부). 데이터셋의 규모가 그리 크지 않기 때문에 학습 과정을 쉽게 실험할 수 있습니다.

Datasets 라이브러리는 허브(hub)에서 데이터셋을 다운로드하고 캐시(cache) 기능을 수행하는 쉬운 명령어를 제공합니다. 다음과 같이 MRPC 데이터셋을 다운로드할 수 있습니다:

In [5]:
from datasets import load_dataset

raw_datasets = load_dataset(
    "glue", "mrpc"
)  #  MRPC 데이터셋은 10개의 데이터셋으로 구성된 GLUE 벤치마크 중 하나.
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})

In [6]:
raw_train_dataset = raw_datasets["train"]
raw_train_dataset.features

{'sentence1': Value(dtype='string', id=None),
 'sentence2': Value(dtype='string', id=None),
 'label': ClassLabel(names=['not_equivalent', 'equivalent'], id=None),
 'idx': Value(dtype='int32', id=None)}

레이블(label)은 ClassLabel 타입이고 레이블 이름에 대한 정수 매핑은 names 폴더에 저장되어 있습니다. 0은 not_equivalent를 의미하고, 1은 equivalent를 나타냅니다.

## 데이터셋 전처리

데이터셋 전처리를 위해서는 우선적으로 텍스트를 모델이 이해할 수 있는 숫자로 변환해야 합니다. 이전 장에서 보았듯이 이는 토크나이저가 담당합니다. 토크나이저에 단일 문장 또는 다중 문장 리스트를 입력할 수 있으므로, 다음과 같이 각 쌍의 모든 첫 번째 문장과 두 번째 문장을 각각 직접 토큰화할 수 있습니다:



In [7]:
from transformers import AutoTokenizer

checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)


def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)


tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
tokenized_datasets

Map: 100%|██████████| 408/408 [00:00<00:00, 11558.38 examples/s]


DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1725
    })
})

마지막으로 해야 할 일은 전체 요소들을 배치(batch)로 분리할 때 가장 긴 요소의 길이로 모든 예제를 채우는(padding) 것입니다. 이를 동적 패딩(dynamic padding)이라고 합니다.

## 동적 패딩(Dynamic padding)

In [8]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

## 학습 (Training)

Trainer를 정의하기 전에 먼저 수행할 단계는 Trainer가 학습 및 평가에 사용할 모든 하이퍼파라미터(hyperparameters)를 포함하는 TrainingArguments 클래스를 정의하는 것입니다. 

In [9]:
from transformers import TrainingArguments
from transformers import AutoModelForSequenceClassification
from transformers import Trainer

# 클래스 정의
training_args = TrainingArguments("test-trainer")
# 모델 정의
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)
# trainer 정의
trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)
# 학습 시작
trainer.train()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss
500,0.500000
1000,0.256800


TrainOutput(global_step=1377, training_loss=0.30944098124230524, metrics={'train_runtime': 38.2818, 'train_samples_per_second': 287.447, 'train_steps_per_second': 35.97, 'total_flos': 405114969714960.0, 'train_loss': 0.30944098124230524, 'epoch': 3.0})

## 평가 (Evaluation)
유용한 compute_metrics() 함수를 구현하고 이를 학습할 때 사용하는 방법을 살펴보겠습니다.

In [14]:
import evaluate
import numpy as np


def compute_metrics(eval_preds):
    metric = evaluate.load("glue", "mrpc")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)


training_args = TrainingArguments("test-trainer", evaluation_strategy="epoch")
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()

/mnt/c4a1c56d-0589-48cd-9717-f6c33a86fa3b/dataScience/learn_LLM/.venv/lib/python3.9/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.460902,0.732843,0.835596
2,0.555500,0.451315,0.877451,0.913495
3,0.368500,0.563403,0.867647,0.905594


TrainOutput(global_step=1377, training_loss=0.40370349426934404, metrics={'train_runtime': 42.747, 'train_samples_per_second': 257.421, 'train_steps_per_second': 32.213, 'total_flos': 405114969714960.0, 'train_loss': 0.40370349426934404, 'epoch': 3.0})

Trainer class안쓰고 같은 결과를 얻는 방법

In [16]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)


def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)


tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
tokenized_datasets = tokenized_datasets.remove_columns(
    ["sentence1", "sentence2", "idx"]
)
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets.set_format("torch")
tokenized_datasets["train"].column_names

from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    tokenized_datasets["train"], shuffle=True, batch_size=8, collate_fn=data_collator
)
eval_dataloader = DataLoader(
    tokenized_datasets["validation"], batch_size=8, collate_fn=data_collator
)

from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

from transformers import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)

from transformers import get_scheduler

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

import evaluate

metric = evaluate.load("glue", "mrpc")
model.eval()
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
  0%|          | 0/1377 [00:57<?, ?it/s]


{'accuracy': 0.8382352941176471, 'f1': 0.89}

# 4장. 모델 및 토크나이저 공유

# 5장. Datasets 라이브러리

## 로컬 데이터셋 로딩하기

여기서는 예제 데이터로 이탈리아어 질의응답(question answering)을 위한 대규모 데이터셋인 SQuAD-it dataset을 사용합니다.

In [4]:
from datasets import load_dataset

data_files = {"train": "../input/drugsComTrain_raw.tsv", "test": "../input/drugsComTest_raw.tsv"}
# \t 는 Python에서 탭 문자를 나타냅니다.
drug_dataset = load_dataset("csv", data_files=data_files, delimiter="\t")


Generating train split: 161297 examples [00:00, 373782.24 examples/s]
Generating test split: 53766 examples [00:00, 361096.00 examples/s]


## 데이터셋 슬라이싱(slicing)과 다이싱(dicing)

Pandas와 유사하게 🤗Datasets는 Dataset 및 DatasetDict 객체의 내용을 조작하는 여러 기능을 제공합니다.


In [5]:
drug_sample = drug_dataset["train"].shuffle(seed=42).select(range(1000))
# 앞쪽의 샘플 몇개를 가져옵니다.
drug_sample[:3]

{'Unnamed: 0': [87571, 178045, 80482],
 'drugName': ['Naproxen', 'Duloxetine', 'Mobic'],
 'condition': ['Gout, Acute', 'ibromyalgia', 'Inflammatory Conditions'],
 'review': ['"like the previous person mention, I&#039;m a strong believer of aleve, it works faster for my gout than the prescription meds I take. No more going to the doctor for refills.....Aleve works!"',
  '"I have taken Cymbalta for about a year and a half for fibromyalgia pain. It is great\r\nas a pain reducer and an anti-depressant, however, the side effects outweighed \r\nany benefit I got from it. I had trouble with restlessness, being tired constantly,\r\ndizziness, dry mouth, numbness and tingling in my feet, and horrible sweating. I am\r\nbeing weaned off of it now. Went from 60 mg to 30mg and now to 15 mg. I will be\r\noff completely in about a week. The fibro pain is coming back, but I would rather deal with it than the side effects."',
  '"I have been taking Mobic for over a year with no side effects other than 

재현성(reproducibility)을 위해 Dataset.shuffle()의 seed를 고정했습니다. Dataset.select()는 반복 가능 인덱스(iterable indices)를 매개변수로 입력해야 하므로 데이터셋에서 처음 1,000개의 예제를 가져오기 위해 range(1000)을 전달했습니다. 이 샘플들에서 우리는 해당 데이터셋의 몇 가지 단점(quirks)을 볼 수 있습니다:

- "Unnamed: 0" 컬럼(column)은 확실하지는 않지만 각 환자의 익명 ID(anonymized ID)처럼 보입니다.
- "condition" 컬럼(column)에는 대문자와 소문자 레이블이 혼합되어 있습니다.
- 리뷰(review)의 길이는 다양하며 Python 줄 구분 기호(\r\n)와 '와 같은 HTML 문자 코드가 혼합되어 있습니다.

🤗Datasets를 사용하여 이러한 문제들을 처리하는 방법을 살펴보겠습니다.

In [6]:
drug_dataset = drug_dataset.rename_column(
    original_column_name="Unnamed: 0", new_column_name="patient_id"
)
# 결측치 제거
drug_dataset = drug_dataset.filter(lambda x: x["condition"] is not None)

def lowercase_condition(example):
    return {"condition": example["condition"].lower()}

drug_dataset = drug_dataset.map(lowercase_condition)
# 소문자화가 제대로 진행되었는지 확인.
drug_dataset["train"]["condition"][:3]

Map: 100%|██████████| 53471/53471 [00:02<00:00, 26483.49 examples/s]


['left ventricular dysfunction', 'adhd', 'birth control']

## 새로운 컬럼(column) 만들기

In [7]:
def compute_review_length(example):
    return {"review_length": len(example["review"].split())}
drug_dataset = drug_dataset.map(compute_review_length)
# 첫 학습 예제를 살펴봅니다.
drug_dataset["train"][0]

Map: 100%|██████████| 53471/53471 [00:01<00:00, 32796.55 examples/s]


{'patient_id': 206461,
 'drugName': 'Valsartan',
 'condition': 'left ventricular dysfunction',
 'review': '"It has no side effect, I take it in combination of Bystolic 5 Mg and Fish Oil"',
 'rating': 9.0,
 'date': 'May 20, 2012',
 'usefulCount': 27,
 'review_length': 17}

In [8]:
# Dataset.filter() 함수를 사용하여 30단어 미만으로 표현된 리뷰를 제거해 보겠습니다.
drug_dataset = drug_dataset.filter(lambda x: x["review_length"] > 30)
print(drug_dataset.num_rows)

Filter: 100%|██████████| 53471/53471 [00:00<00:00, 303639.54 examples/s]

{'train': 138514, 'test': 46108}


In [9]:
# 리뷰에 HTML 문자 코드가 있다는 것입니다. Python의 html 모듈을 사용하여 다음과 같이 이러한 문자를 이스케이프 해제(unescape)할 수 있습니다
import html

drug_dataset = drug_dataset.map(lambda x: {"review": html.unescape(x["review"])},
                                batched=True # 속도를 위해
)

Map: 100%|██████████| 46108/46108 [00:00<00:00, 47460.55 examples/s]


## 검증 집합(validation set) 생성
모델 성능 평가에 사용할 수 있는 평가 집합(test set)이 있지만 개발 중에 이 평가 집합을 그대로 두고 별도의 검증 집합(validation set)을 만드는 것이 좋습니다.

In [10]:
drug_dataset_clean = drug_dataset["train"].train_test_split(train_size=0.8, seed=42)
# 기본 "test" 분할을 "validation"으로 변경함.
drug_dataset_clean["validation"] = drug_dataset_clean.pop("test")
# 'DatasetDict'에 "test" 집합을 추가.
drug_dataset_clean["test"] = drug_dataset["test"]
drug_dataset_clean

DatasetDict({
    train: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length'],
        num_rows: 110811
    })
    validation: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length'],
        num_rows: 27703
    })
    test: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length'],
        num_rows: 46108
    })
})

## Datasets으로 빅데이터 문제를 해결

### pile이 무엇인가?

Pile은 [EleutherAI](https://www.eleuther.ai/)가 대규모 언어 모델을 학습하기 위해 만든 영어 텍스트 말뭉치입니다. 여기에는 학술 논문(scientific articles), GitHub 코드 리포지토리 및 필터링된 웹 텍스트에 이르는 다양한 데이터셋이 포함되어 있습니다. 학습 데이터는 [14GB 청크](https://the-eye.eu/public/AI/pile/)로 제공되며 [개별적인 구성 요소](https://the-eye.eu/public/AI/pile_preliminary_components/)를 각각 다운로드할 수도 있습니다.

In [1]:
from datasets import load_dataset

pubmed_dataset = load_dataset("casinca/PUBMED_title_abstracts_2019_baseline", split="train", streaming=True)
pubmed_dataset

/mnt/c4a1c56d-0589-48cd-9717-f6c33a86fa3b/dataScience/learn_LLM/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


IterableDataset({
    features: Unknown,
    n_shards: 1
})

In [2]:
dataset_head = pubmed_dataset.take(5)
list(dataset_head)

[{'meta': {'pmid': 11409574, 'language': 'eng'},
  'text': 'Epidemiology of hypoxaemia in children with acute lower respiratory infection.\nTo determine the prevalence of hypoxaemia in children aged under 5 years suffering acute lower respiratory infections (ALRI), the risk factors for hypoxaemia in children under 5 years of age with ALRI, and the association of hypoxaemia with an increased risk of dying in children of the same age. Systematic review of the published literature. Out-patient clinics, emergency departments and hospitalisation wards in 23 health centres from 10 countries. Cohort studies reporting the frequency of hypoxaemia in children under 5 years of age with ALRI, and the association between hypoxaemia and the risk of dying. Prevalence of hypoxaemia measured in children with ARI and relative risks for the association between the severity of illness and the frequency of hypoxaemia, and between hypoxaemia and the risk of dying. Seventeen published studies were found that

## 자신만의 데이터셋 만들기

일단은 스킵한다.

## FAISS를 이용한 시맨틱 검색

### 시맨틱 검색을 위한 임베딩 사용하기
1장에서 보았듯이 트랜스포머(Transformer) 기반 언어 모델은 텍스트 내의 각 토큰을 임베딩 벡터(embedding vector) 로 나타냅니다. 개별 임베딩을 "풀링(pooling)"하여 전체 문장, 단락 또는 (경우에 따라) 문서에 대한 벡터 표현을 생성할 수 있습니다. 그런 다음 이러한 임베딩을 사용하여 각 임베딩 사이의 내적 유사도(dot-product similarity), 또는 다른 유사도 메트릭(similarity metric)을 계산하고 가장 많이 겹치는 문서를 반환하여 코퍼스에서 유사 문서 검색을 수행할 수 있습니다.

이 섹션에서는 임베딩을 사용하여 시맨틱 검색 엔진을 개발할 것입니다. 이러한 검색 엔진은 쿼리와 문서의 키워드 매칭을 기반으로 하는 기존 접근 방식에 비해 몇 가지 장점을 제공합니다.

In [1]:
from datasets import load_dataset

data_files = {"train": "../input/drugsComTrain_raw.tsv", "test": "../input/drugsComTest_raw.tsv"}
# \t 는 Python에서 탭 문자를 나타냅니다.
drug_dataset = load_dataset("csv", data_files=data_files, delimiter="\t")

# 결측치 제거
drug_dataset = drug_dataset.filter(lambda x: x["condition"] is not None)

drug_dataset

/mnt/c4a1c56d-0589-48cd-9717-f6c33a86fa3b/dataScience/learn_LLM/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount'],
        num_rows: 160398
    })
    test: Dataset({
        features: ['Unnamed: 0', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount'],
        num_rows: 53471
    })
})

In [2]:
# Dataset.remove_columns() 함수를 사용하여 그외의 나머지 열을 삭제해 보겠습니다:
dataset = drug_dataset["train"]
columns = dataset.column_names
columns_to_keep = ["drugName", "condition"]
columns_to_remove = set(columns_to_keep).symmetric_difference(columns)
dataset = dataset.remove_columns(columns_to_remove)
dataset

Dataset({
    features: ['drugName', 'condition'],
    num_rows: 160398
})

In [3]:
def concatenate_text(examples):
    return {
        "text": examples["drugName"] 
        + " \n " 
        + examples["condition"]
    }

dataset = dataset.map(concatenate_text)

from transformers import AutoTokenizer, AutoModel
import torch

model_ckpt = "sentence-transformers/multi-qa-mpnet-base-dot-v1"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = AutoModel.from_pretrained(model_ckpt)
device = torch.device("cuda")
model.to(device)

def cls_pooling(model_output):
    return model_output.last_hidden_state[:, 0]


def get_embeddings(text_list):
    encoded_input = tokenizer(text_list, padding=True, truncation=True, return_tensors="pt")
    encoded_input = {k: v.to(device) for k, v in encoded_input.items()}
    model_output = model(**encoded_input)
    return cls_pooling(model_output)

embedding = get_embeddings(dataset["text"][0])
embedding.shape

torch.Size([1, 768])

In [4]:
embeddings_dataset = dataset.map(
    lambda x: {"embeddings": get_embeddings(x["text"]).detach().cpu().numpy()[0]}
)
embeddings_dataset.add_faiss_index(column="embeddings")
question = "What is the best drug in each condition?"
question_embedding = get_embeddings([question]).cpu().detach().numpy()
# question_embedding.shape
scores, samples = embeddings_dataset.get_nearest_examples("embeddings", question_embedding, k=5)

import pandas as pd

samples_df = pd.DataFrame.from_dict(samples)
samples_df["scores"] = scores
samples_df.sort_values("scores", ascending=False, inplace=True)

for _, row in samples_df.iterrows():
    print(f"COMMENT: {row.comments}")
    print(f"SCORE: {row.scores}")
    print(f"TITLE: {row.title}")
    print(f"URL: {row.html_url}")
    print("=" * 50)
    print()


100%|██████████| 161/161 [00:00<00:00, 408.53it/s]


AttributeError: 'Series' object has no attribute 'comments'

# Tokenizers 라이브러리

- 새로운 텍스트 말뭉치를 기반으로 특정 체크포인트의 토크나이저와 유사한 새로운 토크나이저를 학습시키는 방법
- 오늘날 NLP에서 사용되는 세 가지 주요 단어 토큰화 알고리즘의 차이점
- 🤗Tokenizers 라이브러리를 사용하여 처음부터 토크나이저를 구축하고 특정 데이터로 학습하는 방법



## 기존 토크나이저에서 새로운 토크나이저 학습

In [1]:
from datasets import load_dataset

# 로드하는데 몇 분이 소요될 수 있습니다. 커피나 차를 준비하세요.
raw_datasets = load_dataset("code_search_net", "python")

# 이제 텍스트 배치(batch)의 이터레이터(iterator) 형태로 말뭉치를 구성
def get_training_corpus():
    return (
        raw_datasets["train"][i : i + 1000]["whole_func_string"]
        for i in range(0, len(raw_datasets["train"]), 1000)
    )

training_corpus = get_training_corpus()

# 모델과 일치시키려는 토크나이저를 로드해야
from transformers import AutoTokenizer

old_tokenizer = AutoTokenizer.from_pretrained("gpt2")

# train_new_from_iterator() 사용 중인 토크나이저가 "빠른(fast)" 토크나이저인 경우에만 작동합
tokenizer = old_tokenizer.train_new_from_iterator(training_corpus, 52000)
# tokenizer.save_pretrained("code-search-net-tokenizer")

/mnt/c4a1c56d-0589-48cd-9717-f6c33a86fa3b/dataScience/learn_LLM/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
example = '''def add_numbers(a, b):
    """Add the two numbers `a` and `b`."""
    return a + b'''

old_tokens = old_tokenizer.tokenize(example)
new_tokens = tokenizer.tokenize(example)

print("Old tokenizer result:")
print(old_tokens)
print("\nNew tokenizer result:")
print(new_tokens)

Old tokenizer result:
['def', 'Ġadd', '_', 'n', 'umbers', '(', 'a', ',', 'Ġb', '):', 'Ċ', 'Ġ', 'Ġ', 'Ġ', 'Ġ"""', 'Add', 'Ġthe', 'Ġtwo', 'Ġnumbers', 'Ġ`', 'a', '`', 'Ġand', 'Ġ`', 'b', '`', '."', '""', 'Ċ', 'Ġ', 'Ġ', 'Ġ', 'Ġreturn', 'Ġa', 'Ġ+', 'Ġb']

New tokenizer result:
['def', 'Ġadd', '_', 'numbers', '(', 'a', ',', 'Ġb', '):', 'ĊĠĠĠ', 'Ġ"""', 'Add', 'Ġthe', 'Ġtwo', 'Ġnumbers', 'Ġ`', 'a', '`', 'Ġand', 'Ġ`', 'b', '`."""', 'ĊĠĠĠ', 'Ġreturn', 'Ġa', 'Ġ+', 'Ġb']


## "빠른(fast)" 토크나이저의 특별한 능력

### 배치 인코딩 (Batch encoding)

### 입력(inputs)에서 예측(predictions)까지

### 엔터티 그룹화

## QA 파이프라인에서의 "빠른(fast)" 토크나이저

질의 응답(question answering) 작업에 관심이 없다면 이 섹션을 건너뛸 수 있습니다.

## 정규화(Normalization) 및 사전 토큰화(Pre-tokenization)

텍스트를 하위 토큰(subtokens)으로 분할하기 전에(모델에 따라), 토크나이저는 정규화(normalization) 및 사전 토큰화(pre-tokenization) 두 단계를 수행합니다.

### 정규화(Normalization)
정규화 단계에는 불필요한 공백 제거, 소문자 변환(lowercasing) 및 악센트 제거 등과 같은 몇가지 일반적인 정제 작업이 포함됩니다.

### 사전토큰화(Pre-tokenization)

토크나이저는 원시 텍스트만으로는 학습될 수 없습니다. 대신에 먼저 텍스트를 단어와 같은 작은 개체들로 분할해야 합니다. 여기서 사전 토큰화(pre-tokenization) 단계가 실행됩니다. 

### SentencePiece

텍스트 전처리를 위한 토큰화 알고리즘입니다. 텍스트를 일련의 유니코드 문자들로 간주하고 공백을 특수 문자인 _로 치환합니다. Unigram 알고리즘(섹션 7 참조)과 함께 사용하면 사전 토큰화(pre-tokenization) 단계가 필요하지 않으므로 공백 문자가 사용되지 않는 언어(예: 중국어 또는 일본어)에 매우 유용합니다.

## Byte-Pair Encoding (BPE) 토큰화

 토큰화 알고리즘에 대한 일반적인 개요만을 원하는 경우 이 장을 건너뛰어도 됩니다.

## WordPiece 토큰화

 WordPiece를 심층적으로 다루며 전체 구현 과정을 보여줍니다. 토큰화 알고리즘에 대한 일반적인 개요를 원하는 경우 생략해도 좋습니다.

 ## Unigram 토큰화

  토큰화 알고리즘에 대한 일반적인 개요를 원하는 경우 생략해도 좋습니다.

## 블록 단위로 토크나이저 빌딩하기


 토큰화는 다음과 같은 단계로 실행됩니다:

- 정규화 (공백이나 악센트 제거, 유니코드 정규화 등과 같이 필요하다고 여겨지는 모든 텍스트 정제 작업)
- 사전 토큰화 (입력을 단어들로 분리)
- 모델을 통한 입력 실행 (사전 토큰화된 단어들을 사용하여 토큰 시퀀스 생성)
- 후처리 (토큰나이저의 특수 토큰 추가, attention mask 및 토큰 유형 ID 생성)


### 말뭉치 확보

새로운 토크나이저를 학습하기 위해 작은 텍스트 말뭉치를 사용합니다


In [3]:
from datasets import load_dataset


dataset = load_dataset("wikitext", name="wikitext-2-raw-v1", split="train")


def get_training_corpus():
    for i in range(0, len(dataset), 1000):
        yield dataset[i : i + 1000]["text"]

# WordPiece 토크나이저를 처음부터 빌딩하기
from tokenizers import (
    decoders,
    models,
    normalizers,
    pre_tokenizers,
    processors,
    trainers,
    Tokenizer,
)

tokenizer = Tokenizer(models.WordPiece(unk_token="[UNK]"))
# 토큰화의 첫 번째 단계는 정규화(normalization)입니다.
tokenizer.normalizer = normalizers.Sequence(
    [normalizers.NFD(), normalizers.Lowercase(), normalizers.StripAccents()]
)
# 다음은 사전 토큰화 단계입니다
tokenizer.pre_tokenizer = pre_tokenizers.Whitespace()

# 모든 특수 토큰을 전달해야 한다는 것
special_tokens = ["[UNK]", "[PAD]", "[CLS]", "[SEP]", "[MASK]"]
trainer = trainers.WordPieceTrainer(vocab_size=25000, special_tokens=special_tokens)

# 앞에서 정의한 반복자(iterator)를 사용하여 모델을 학습
tokenizer.train_from_iterator(get_training_corpus(), trainer=trainer)

# 테스트
encoding = tokenizer.encode("Let's test this tokenizer")
print(encoding.tokens)



Generating validation split: 100%|██████████| 3760/3760 [00:00<00:00, 1183711.10 examples/s]





['let', "'", 's', 'test', 'this', 'tok', '##eni', '##zer']


# 주요 NLP 태스크 실제 구현방법

다음과 같은 주요 NLP 태스크들을 다룰 것입니다.
- 토큰 분류 (Token Classification)
- 마스킹된 언어 모델링 (Masked Language Modeling)
- 요약 (Summarization)
- 번역 (Translation)
- 인과적 언어 모델링 사전학습 (Causal Language Modeling Pretraining like GPT-2)
- 질의응답 (Question Answering)

## 토큰 분류 (Token Classification)

이 포괄적인 작업은 다음과 같이 "문장의 각 토큰에 레이블을 지정"하는 것으로 정형화될 수 있는 모든 문제를 포함

In [9]:
from datasets import load_dataset

raw_datasets = load_dataset("conll2003", trust_remote_code=True)

# 데이터 처리
from transformers import AutoTokenizer

model_checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # 새로운 단어의 시작 토큰.
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # 특수 토큰.
            new_labels.append(-100)
        else:
            # 이전 토큰과 동일한 단어에 소속된 토큰.
            label = labels[word_id]
            # 만약 레이블이 B-XXX이면 이를 I-XXX로 변경.
            if label % 2 == 1:
                label += 1
            new_labels.append(label)

    return new_labels

# examples는 단일 텍스트(문장)가 아니라 다중 텍스트임.
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True
    )
    all_labels = examples["ner_tags"]
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)    # 배치(batch) 인덱스 지정
        new_labels.append(align_labels_with_tokens(labels, word_ids))

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

tokenized_datasets = raw_datasets.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=raw_datasets["train"].column_names,
)

# Trainer API를 이용하여 모델 미세조정

from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)


Map: 100%|██████████| 3453/3453 [00:00<00:00, 26093.16 examples/s]


### 평가 기준 (Metrics) 

이후 생략

## 마스크 언어 모델(Masked Language Model) 미세조정

분야 특화 데이터에 대해 사전 학습된 언어 모델을 미세 조정하는 이 프로세스를 일반적으로 도메인 어뎁테이션(domain adaptation)이라고 합니다. 

### 마스크 언어 모델링(MLM)을 위해 사전학습된 모델(pretrained model) 선택하기



In [10]:
from transformers import AutoModelForMaskedLM

model_checkpoint = "distilbert-base-uncased"
model = AutoModelForMaskedLM.from_pretrained(model_checkpoint)

# 마스크 토큰을 예측하려면 모델에 대한 입력을 생성하기 위해 DistilBERT의 토크나이저가 필요
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

#데이터셋
from datasets import load_dataset

imdb_dataset = load_dataset("imdb")

# 데이터 전처리
def tokenize_function(examples):
    result = tokenizer(examples["text"])
    if tokenizer.is_fast:
        result["word_ids"] = [result.word_ids(i) for i in range(len(result["input_ids"]))]
    return result


# 빠른 멀티스레딩을 작동시키기 위해서, batched=True를 지정합니다.
tokenized_datasets = imdb_dataset.map(
    tokenize_function, batched=True, remove_columns=["text", "label"]
)

# 메모리에 들어갈 수 있는 약간 작은 수치를 선택
chunk_size = 128

def group_texts(examples):
    # 모든 텍스트들을 결합한다.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    # 결합된 텍스트들에 대한 길이를 구한다.
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # `chunk_size`보다 작은 경우 마지막 청크를 삭제
    total_length = (total_length // chunk_size) * chunk_size
    # max_len 길이를 가지는 chunk 단위로 슬라이스
    result = {
        k: [t[i : i + chunk_size] for i in range(0, total_length, chunk_size)]
        for k, t in concatenated_examples.items()
    }
    # 새로운 레이블 컬럼을 생성
    result["labels"] = result["input_ids"].copy()
    return result                            

lm_datasets = tokenized_datasets.map(group_texts, batched=True)

# Trainer API를 이용하여 DistilBERT 미세조정
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

Map: 100%|██████████| 50000/50000 [00:40<00:00, 1249.17 examples/s]


In [23]:
train_size = 50_000
test_size = int(0.1 * train_size)

downsampled_dataset = lm_datasets["train"].train_test_split(
    train_size=train_size, test_size=test_size, seed=42
)
downsampled_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 50000
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 5000
    })
})

In [24]:
from transformers import TrainingArguments

batch_size = 64
# Show the training loss with every epoch
logging_steps = len(downsampled_dataset["train"]) // batch_size
model_name = model_checkpoint.split("/")[-1]

training_args = TrainingArguments(
    # output_dir=f"{model_name}-finetuned-imdb",
    # overwrite_output_dir=True,
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    # push_to_hub=True,
    fp16=True,
    logging_steps=logging_steps,
)

from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=downsampled_dataset["train"],
    eval_dataset=downsampled_dataset["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

import math

eval_results = trainer.evaluate()
print(f">>> Perplexity: {math.exp(eval_results['eval_loss']):.2f}")
trainer.train()

eval_results  = trainer.evaluate()
print(f">>> Perplexity: {math.exp(eval_results['eval_loss']):.2f}")


/mnt/c4a1c56d-0589-48cd-9717-f6c33a86fa3b/dataScience/learn_LLM/.venv/lib/python3.9/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


>>> Perplexity: 11.06


Epoch,Training Loss,Validation Loss,Model Preparation Time
1,2.496500,2.333867,0.001200
2,2.424800,2.296707,0.001200
3,2.391800,2.276380,0.001200


>>> Perplexity: 9.97


## 번역

번역(translation)에 대해서 알아봅시다. 이것은 또 다른 형태의 sequence-to-sequence 태스크입니다. 즉, 한 시퀀스에서 다른 시퀀스로 이동하는(변형하는) 것이죠. 그런 의미에서 이 문제는 요약(summarization)과 매우 유사하고 여기에서 보게 될 내용을 다음과 같은 다른 sequence-to-sequence 문제에 적용할 수 있습니다:

- 스타일 트랜스퍼(style transfer): 특정 스타일로 작성된 텍스트를 다른 스타일로 번역하는 모델 생성(예: 격식 스타일에서 캐주얼 스타일로 또는 셰익스피어 영어에서 현대 영어로)
- 생성 기반 질의 응답(generative question answering): 주어진 맥락(context)에서 질문에 대한 답변을 생성하는 모델 만들기

두 개(또는 그 이상) 언어로 된 충분히 큰 텍스트 코퍼스가 있는 경우 인과적 언어 모델링(causal language modeling) 섹션에서 하는 것처럼 처음부터 새로운 번역 모델을 학습할 수 있습니다. 그러나 일정 규모의 특정 언어로 표현된 말뭉치를 가지고 mT5나 mBART 등과 같은 이미 존재하는 다국어 번역 모델을 미세 조정하는 편이 훨씬 빠를 겁니다.

### 데이터 준비

번역 모델을 미세 조정하거나 처음부터 사전 학습하려면 작업에 적합한 데이터셋이 필요합니다. KDE4 데이터셋을 `load_dataset()` 함수를 사용하여 데이터셋을 다운로드합니다:

In [1]:
from datasets import load_dataset
# from evaluate import load_metric

raw_datasets = load_dataset("kde4", lang1="en", lang2="fr")
split_datasets = raw_datasets["train"].train_test_split(train_size=0.9, seed=42)
split_datasets["validation"] = split_datasets.pop("test")
split_datasets["train"][1]["translation"]

/mnt/c4a1c56d-0589-48cd-9717-f6c33a86fa3b/dataScience/learn_LLM/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


{'en': 'Publisher', 'fr': 'Éditeur'}

### 데이터 처리

모든 텍스트는 모델이 이해할 수 있도록 토큰 ID 세트로 변환되어야 합니다. 이 작업을 위해 입력과 타겟을 모두 토큰화해야 합니다. 첫 번째 작업은 tokenizer 객체를 만드는 것입니다. 앞서 언급했듯이 Marian English to French 사전 학습 모델을 사용할 것입니다. 다른 언어 쌍에 대해서 작업하려면 모델 체크포인트를 변경해야 합니다. Helsinki-NLP 조직은 여러 언어로 천 개 이상의 모델을 제공합니다.

In [2]:
from transformers import AutoTokenizer

model_checkpoint = "Helsinki-NLP/opus-mt-en-fr"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, return_tensors="pt")
# mBART, mBART-50 또는 M2M100과 같은 다국어 토크나이저를 사용하는 경우 tokenizer.src_lang 및 tokenizer.tgt_lang을 올바른 값으로 설정하여 토크나이저에서 입력 및 대상의 언어 코드를 설정해야 합니다.

max_input_length = 128
max_target_length = 128


def preprocess_function(examples):
    inputs = [ex["en"] for ex in examples["translation"]]
    targets = [ex["fr"] for ex in examples["translation"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # 타겟을 위한 토크나이저 셋업
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_datasets = split_datasets.map(
    preprocess_function,
    batched=True,
    remove_columns=split_datasets["train"].column_names,
)


/mnt/c4a1c56d-0589-48cd-9717-f6c33a86fa3b/dataScience/learn_LLM/.venv/lib/python3.9/site-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
Map:   0%|          | 0/189155 [00:00<?, ? examples/s]/mnt/c4a1c56d-0589-48cd-9717-f6c33a86fa3b/dataScience/learn_LLM/.venv/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:4109: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Map: 100%|██████████| 21018/21018 [00:01<00:00, 17995.96 examples/s]


이제 데이터가 전처리되었으므로 사전 학습된 모델을 미세 조정할 준비가 되었습니다!

### Trainer API로 모델 미세 조정하기

여기에서는 Seq2SeqTrainer를 사용합니다. Seq2SeqTrainer는 Trainer의 하위 클래스로서 입력에 대한 출력을 예측하기 위해 generate() 메서드를 사용하여 평가를 적절하게 수행할 수 있습니다.

먼저 미세 조정할 실제 모델이 필요합니다. 일반적인 AutoModel API를 사용합니다:

In [3]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

### 데이터 콜레이션 (Data Collation)
동적 배치 처리(Dynamic batching)를 위한 패딩을 처리하려면 데이터 콜레이터가 필요합니다. 

In [4]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)


다음으로 평가 지표(metric)를 살펴보겠습니다.

### 평가지표 (Metrics)

Seq2SeqTrainer가 수퍼클래스 Trainer에 추가하는 기능은 평가(evaluation) 또는 예측(prediction) 중에 generate() 메서드를 사용하는 기능입니다.

In [8]:
import evaluate
import numpy as np

metric = evaluate.load("sacrebleu")

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    # 모델이 예측 로짓(logits)외에 다른 것을 리턴하는 경우.
    if isinstance(preds, tuple):
        preds = preds[0]

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # -100은 건너뛴다.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # 단순 후처리
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    return {"bleu": result["score"]}

from transformers import Seq2SeqTrainingArguments

args = Seq2SeqTrainingArguments(
    f"marian-finetuned-kde4-en-to-fr",
    evaluation_strategy="no",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=True,
    # push_to_hub=True,
)

/mnt/c4a1c56d-0589-48cd-9717-f6c33a86fa3b/dataScience/learn_LLM/.venv/lib/python3.9/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


마지막으로 모든 것을 trainer로 전달합니다.

In [9]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.evaluate(max_length=max_target_length)

{'eval_loss': 1.697405457496643,
 'eval_model_preparation_time': 0.0014,
 'eval_bleu': 39.41460649639242,
 'eval_runtime': 340.641,
 'eval_samples_per_second': 61.701,
 'eval_steps_per_second': 0.966}

In [10]:
trainer.train()
trainer.evaluate(max_length=max_target_length)

Step,Training Loss
500,1.383000
1000,1.243000
1500,1.167100
2000,1.132100
2500,1.101000
3000,1.075500
3500,1.050100
4000,1.043800
4500,1.008600
5000,1.025300


/mnt/c4a1c56d-0589-48cd-9717-f6c33a86fa3b/dataScience/learn_LLM/.venv/lib/python3.9/site-packages/transformers/modeling_utils.py:2618: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[59513]]}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


{'eval_loss': 0.8628926873207092,
 'eval_model_preparation_time': 0.0014,
 'eval_bleu': 53.21851308777082,
 'eval_runtime': 355.5299,
 'eval_samples_per_second': 59.117,
 'eval_steps_per_second': 0.925,
 'epoch': 3.0}

## 요약 (Summarization)

Transformer 모델을 사용하여 긴 문서를 간략하게 압축하는 방법, 즉 텍스트 요약(text summarization) 태스크를 살펴보겠습니다. 이 작업은 가장 어려운 NLP 작업 중 하나로 알려져 있는데 그 이유는 길이가 긴 구절을 이해하고 전체 문서의 핵심 주제를 포괄하는 일관된 텍스트를 생성하는 등 다양한 능력이 필요하기 때문입니다.

### 다중 언어 말뭉치 준비하기

Multilingual Amazon Reviews Corpus를 사용하여 이중 언어 요약기를 생성합니다. 이 말뭉치는 6개 언어로 된 Amazon 제품 리뷰로 구성되며 일반적으로 다국어 분류기를 벤치마킹하는 데 사용됩니다. 그러나 각 리뷰에는 짧은 제목이 수반되므로 이 제목을 모델이 학습할 대상 요약으로 사용할 수 있습니다

In [11]:
from datasets import load_dataset

spanish_dataset = load_dataset("amazon_reviews_multi", "es")
english_dataset = load_dataset("amazon_reviews_multi", "en")
# english_dataset
# 우리가 관심 있는 리뷰 정보는 review_body 및 review_title 열에 포함되어 있습니다. 
# 5장에서 배운 기법을 사용하여 학습 집합에서 무작위 샘플을 가져오는 간단한 함수를 만들어 보죠.

def show_samples(dataset, num_samples=3, seed=42):
    sample = dataset["train"].shuffle(seed=seed).select(range(num_samples))
    for example in sample:
        print(f"\n'>> Title: {example['review_title']}'")
        print(f"'>> Review: {example['review_body']}'")

# show_samples(english_dataset)
# Amazon의 대표적인 테마 상품에 집중하기 위해 서평 요약(book review)에 집중합시다.
def filter_books(example):
    return (
        example["product_category"] == "book"
        or example["product_category"] == "digital_ebook_purchase"
    )

# 필터를 적용하기 전에 english_dataset 형식을 "pandas"에서 "arrow"로 다시 전환
english_dataset.reset_format()

spanish_books = spanish_dataset.filter(filter_books)
english_books = english_dataset.filter(filter_books)

# 두 개의 Dataset 객체를 합치는 concatenate_datasets() 함수를 사용합니다.
from datasets import concatenate_datasets, DatasetDict

books_dataset = DatasetDict()

for split in english_books.keys():
    books_dataset[split] = concatenate_datasets(
        [english_books[split], spanish_books[split]]
    )
    books_dataset[split] = books_dataset[split].shuffle(seed=42)

# 몇 개의 샘플을 선택합니다.
show_samples(books_dataset)


DefunctDatasetError: Dataset 'amazon_reviews_multi' is defunct and no longer accessible due to the decision of data providers

데이터가 더이상 다운로드가 어렵기 때문에 아래 내용은 더이상 진행하지 않음.

### 데이터 전처리하기

다음 작업은 리뷰와 제목을 토큰화하고 인코딩하는 것입니다. 평소처럼 사전 학습된 모델 체크포인트와 연결된 토크나이저를 로드하는 것으로 시작합니다. 가급적 짧은 시간 내에 모델을 미세 조정할 수 있도록 mt5-small을 체크포인트로 사용할 것입니다:


In [ ]:
from transformers import AutoTokenizer

model_checkpoint = "google/mt5-small"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

max_input_length = 512
max_target_length = 30

# 모델에 지나치게 긴 입력을 전달하지 않도록 리뷰와 제목 모두에 절단 작업(truncation)을 수행
def preprocess_function(examples):
    model_inputs = tokenizer(
        examples["review_body"], max_length=max_input_length, truncation=True
    )
    # 타겟을 위한 토크나이저 설정
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            examples["review_title"], max_length=max_target_length, truncation=True
        )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_datasets = books_dataset.map(preprocess_function, batched=True)


### 텍스트 요약을 위한 평가지표(metrics)

이 코스에서 다룬 대부분의 다른 작업과 비교할 때 요약 또는 번역과 같은 텍스트 생성 작업의 성능을 측정하는 것은 간단하지 않습니다.

In [ ]:
# !uv add rouge_score
import evaluate
rouge_score = evaluate.load("sacrebleu")

## 인과적 언어 모델(Causal Language Model)을 처음부터 학습하기

코드 생성 모델의 축소 버전을 구축할 것입니다. Python 코드의 하위 집합을 사용하여 전체 함수나 클래스 대신 한 줄 완성(one-line completions)에 중점을 둘 것입니다. 

### 데이터 모으기
Python 코드는 GitHub와 같은 코드 리포지토리에서 풍부하게 제공되며, 모든 Python 리포지토리를 스크랩하여 데이터셋을 만드는 데 사용할 수 있습니다.


In [1]:
# from collections import defaultdict
# from tqdm import tqdm
# from datasets import Dataset

# def any_keyword_in_string(string, keywords):
#     for keyword in keywords:
#         if keyword in string:
#             return True
#     return False

# def filter_streaming_dataset(dataset, filters):
#     filtered_dict = defaultdict(list)
#     total = 0
#     for sample in tqdm(iter(dataset)):
#         total += 1
#         if any_keyword_in_string(sample["content"], filters):
#             for k, v in sample.items():
#                 filtered_dict[k].append(v)
#     print(f"{len(filtered_dict['content'])/total:.2%} of data after filtering.")
#     return Dataset.from_dict(filtered_dict)

# 전체 데이터 세트를 필터링하는 데 컴퓨터와 대역폭에 따라 2-3시간이 걸릴 수 있습니다. 이 많은 시간이 걸리는 프로세스를 직접 수행하고 싶지 않다면 허브에서 필터링된 데이터셋을 직접 다운로드할 수 있습니다:
from datasets import load_dataset, DatasetDict

ds_train = load_dataset("huggingface-course/codeparrot-ds-train", split="train")
ds_valid = load_dataset("huggingface-course/codeparrot-ds-valid", split="validation")

raw_datasets = DatasetDict(
    {
        "train": ds_train.shuffle().select(range(50000)),
        "valid": ds_valid.shuffle().select(range(500))
    }
)

raw_datasets

/mnt/c4a1c56d-0589-48cd-9717-f6c33a86fa3b/dataScience/learn_LLM/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


DatasetDict({
    train: Dataset({
        features: ['repo_name', 'path', 'copies', 'size', 'content', 'license'],
        num_rows: 50000
    })
    valid: Dataset({
        features: ['repo_name', 'path', 'copies', 'size', 'content', 'license'],
        num_rows: 500
    })
})

### 데이터셋 준비하기
첫 번째 단계는 데이터를 토큰화하여 학습에 사용할 수 있도록 하는 것입니다. 

In [2]:
from transformers import AutoTokenizer

context_length = 128
tokenizer = AutoTokenizer.from_pretrained("huggingface-course/code-search-net-tokenizer")

def tokenize(element):
    outputs = tokenizer(
        element['content'],
        truncation=True,
        max_length=context_length,
        return_overflowing_tokens=True,
        return_length=True,
    )
    input_batch = []
    for length, input_ids in zip(outputs['length'], outputs['input_ids']):
        if length == context_length:
            input_batch.append(input_ids)
    return {"input_ids": input_batch}


tokenized_datasets = raw_datasets.map(
    tokenize, batched=True, remove_columns=raw_datasets["train"].column_names)
tokenized_datasets


Map: 100%|██████████| 500/500 [00:00<00:00, 809.22 examples/s]


DatasetDict({
    train: Dataset({
        features: ['input_ids'],
        num_rows: 1379373
    })
    valid: Dataset({
        features: ['input_ids'],
        num_rows: 12754
    })
})

이제 데이터셋이 준비되었으므로 모델을 설정하겠습니다. 

### 새로운 모델의 초기화
첫 번째 단계는 GPT-2 모델을 새로 초기화하는 것입니다.

In [3]:
from transformers import AutoTokenizer, GPT2LMHeadModel, AutoConfig

config = AutoConfig.from_pretrained(
    "gpt2",
    vocab_size=len(tokenizer),
    n_ctx=context_length,
    bos_token_id=tokenizer.bos_token_id,
    eos_token_id=tokenizer.eos_token_id
)

# 해당 설정으로 새 모델을 로드할 수 있습니다. 
# 실제로 모델을 직접 초기화하기 때문에 from_pretrained() 함수를 사용하지 않음

model = GPT2LMHeadModel(config)
model_size = sum(t.numel() for t in model.parameters())
print(f"GPT-2 size: {model_size/1000**2:.1f}M parameters")

GPT-2 size: 124.2M parameters


학습을 시작하기 전에 배치 생성을 처리할 데이터 콜레이터를 설정해야 합니다. 언어 모델링을 위해 특별히 설계된 DataCollatorForLanguageModeling 콜레이터를 사용할 수 있습니다.

In [4]:
from transformers import DataCollatorForLanguageModeling

tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

이제 모델을 실제로 학습할 수 있게 모든 것이 준비되었습니다.

In [5]:
from transformers import Trainer, TrainingArguments

args = TrainingArguments(
    output_dir="codeparrot-ds",
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    eval_strategy="steps",
    eval_steps=5000,
    logging_steps=5000,
    gradient_accumulation_steps=8,
    num_train_epochs=1,
    weight_decay=0.1,
    warmup_steps=1000,
    lr_scheduler_type="cosine",
    learning_rate=5e-4,
    save_steps=5000,
    fp16=True,
    # push_to_hub=True,
)

trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    args=args,
    data_collator=data_collator,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["valid"],
)

# 20시간 걸릴 수 있음
trainer.train()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Step,Training Loss,Validation Loss
5000,2.482400,1.622585


TrainOutput(global_step=5388, training_loss=2.4245188825645534, metrics={'train_runtime': 2924.4282, 'train_samples_per_second': 471.673, 'train_steps_per_second': 1.842, 'total_flos': 9.0101853978624e+16, 'train_loss': 2.4245188825645534, 'epoch': 0.9999536027467174})

학습된 모델이 실제로 얼마나 잘 작동하는지 봅시다! 로그에서 손실이 꾸준히 감소했음을 알 수 있지만 모델을 테스트하기 위해 특정 프롬프트에서 얼마나 잘 작동하는지 살펴보겠습니다.

In [8]:
import torch
from transformers import pipeline, logging

# Set logging to error level to suppress warnings
logging.set_verbosity_error()

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

# Create the pipeline
pipe = pipeline(
    "text-generation", model="huggingface-course/codeparrot-ds", device=device
)

# Set pad_token_id explicitly
pipe.model.config.pad_token_id = pipe.model.config.eos_token_id
pipe.tokenizer.pad_token = pipe.tokenizer.eos_token

txt = """\
# create some data
x = np.random.randn(100)
y = np.random.randn(100)

# create scatter plot with x, y
"""

# Generate text
generated_text = pipe(txt, num_return_sequences=1)[0]["generated_text"]
print(generated_text)

# create some data
x = np.random.randn(100)
y = np.random.randn(100)

# create scatter plot with x, y
plot(x, y, "o", label="data")
